# Linear Regression (Advanced Example)

Objective: Basically what we do here is examine a dataset with E-commerce Customer Data for a company's website and mobile application. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

In [6]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('linear_regression_adv').getOrCreate()

# If you're getting an error with numpy, please type 'sudo pip install numpy --user' into the EC2 console.
from pyspark.ml.regression import LinearRegression

In [7]:
# Use Spark to read in the Ecommerce Customers csv file. You can infer csv schemas. 
data = spark.read.csv("Datasets/ecommerce_data.csv",inferSchema=True,header=True)

In [8]:
# Print the schema of the DataFrame. You can see potential features as well as the predictor.
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
# Let's focus on one row to make it easier to read.
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [10]:
# A simple for loop allows us to make it even clearer. 
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up a DataFrame for Machine Learning (MLlib)

We need to do a few things before Spark can accept the data for machine learning. First of all, it needs to be in the form of two columns: label and features. Unlike the documentation example, this data is messy. We'll need to combine all of the features into a single vector. VectorAssembler simplifies the process.

In [11]:
# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [12]:
# The input columns are the feature column names, and the output column is what you'd like the new column to be named. 
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [13]:
# Now that we've created the assembler variable, let's actually transform the data.
output = assembler.transform(data)

In [14]:
# Using print schema, you see that the features output column has been added. 
output.printSchema()

# You can see that the features column is a dense vector that combines the various features as expected.
output.head(1)

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
# Let's select two columns (the feature and predictor).
# This is now in the appropriate format to be processed by Spark.
final_data = output.select("features",'Yearly Amount Spent')
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [16]:
# Let's do a randomised 70/30 split. 
# Remember, you can use other splits depending on how easy/difficult it is to train your model.
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [17]:
# Let's see our training data.
train_data.describe().show()

# And our testing data.
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                360|
|   mean|  498.9274690946923|
| stddev|  77.40182366749315|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                140|
|   mean| 500.30807325147305|
| stddev|  84.31484894851235|
|    min| 298.76200786180766|
|    max|  765.5184619388373|
+-------+-------------------+



Now we can create a Linear Regression Model object. Because the feature column is named 'features', we don't have to worry about it. However, as the labelCol isn't the default name, we have to specify it's name (Yearly Amount Spent).

In [18]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [19]:
# Fit the model to the data.
lrModel = lr.fit(train_data)

In [20]:
# Print the coefficients and intercept for linear regression.
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [25.41927300556132,38.761287489448755,-0.24834431402208781,61.93683245411885] Intercept: -1018.0035986586126


In [21]:
# Let's evaluate the model against the test data.
test_results = lrModel.evaluate(test_data)

In [22]:
# Interesting results! This shows the difference between the predicted value and the test data.
test_results.residuals.show()

# Let's get some evaluation metrics (as discussed in the previous linear regression notebook).
print("RSME: {}".format(test_results.rootMeanSquaredError))

+-------------------+
|          residuals|
+-------------------+
|-12.185149077473056|
| -7.946130220394025|
|  10.09146991108264|
| -5.079190159525069|
| 20.551190971066944|
| 2.2269314609062576|
|  3.311487854145639|
|  3.565112293058405|
|  1.105345182549172|
| -4.323573424948563|
|  3.171336826515642|
| -7.375922771416413|
| -5.804961610697205|
| -15.14609917392795|
|  7.109414216357152|
|-17.925257827875214|
| 7.5174508355849525|
|  -2.09893034929172|
|-18.390554114147108|
| -2.921372071008477|
+-------------------+
only showing top 20 rows

RSME: 9.628816937008692


In [23]:
# We can also get the R2 value. 
print("R2: {}".format(test_results.r2))

R2: 0.9868643683094813


Looking at RMSE and R2, we can see that the model is quite accurate. The RMSE shows that, on average, there's only a \$10 discrepancy between the actual and predicted results. Comparing this to the table below, the average amount spent (\$499) and standard deviation (\$79), a \$10 error is surprisingly good. 

The R2 also shows that the model accounts for 98% of the variance in the data. 

In [24]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



## But what if we didn't have the predictor data?

This isn't really relevant to your assignment, but useful in a real-world scenario. What if you have all of these features but no predictor data? How do you actually use the model you've created? Check out the example below.

In [25]:
# Let's just select the features column (removing the label column).
unlabeled_data = test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[31.1280900496166...|
|[31.1695067987115...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.3662121671876...|
|[31.3895854806643...|
|[31.4252268808548...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5257524169682...|
|[31.5741380228732...|
|[31.6548096756927...|
|[31.8164283341993...|
|[31.8512531286083...|
|[31.9120759292006...|
|[31.9563005605233...|
|[31.9673209478824...|
+--------------------+
only showing top 20 rows



In [26]:
# Now we can transform the unlabeled data.
predictions = lrModel.transform(unlabeled_data)

In [27]:
# It worked! Feeding the unlabeled data features into the model results in a prediction, 
# which is the amount someone with those features is likely to spend in a year.
predictions.show()
predictions.head(1)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|332.11401888066666|
|[31.1280900496166...| 565.1988169674487|
|[31.1695067987115...|417.26506089121017|
|[31.2681042107507...|  428.549723333349|
|[31.2834474760581...| 571.2298984546005|
|[31.3091926408918...|430.49378637902737|
|[31.3584771924370...|491.86446259532977|
|[31.3662121671876...| 427.0237702634265|
|[31.3895854806643...| 408.9642658774337|
|[31.4252268808548...| 535.0902920797105|
|[31.4459724827577...| 481.7056281086129|
|[31.4474464941278...|425.97866486664043|
|[31.5257524169682...| 449.7705884205791|
|[31.5741380228732...| 559.5553713345148|
|[31.6548096756927...|468.15400951119136|
|[31.8164283341993...| 519.0477493315316|
|[31.8512531286083...|465.47479583121344|
|[31.9120759292006...|389.63364665499944|
|[31.9563005605233...| 565.5164858613459|
|[31.9673209478824...|448.67121331066073|
+--------------------+------------

[Row(features=DenseVector([30.3932, 11.803, 36.3158, 2.0838]), prediction=332.11401888066666)]